In [3]:
#######################################################################
# Imports
#######################################################################
import numpy as np

# QUBO / Ising
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.translators import to_ising

# Custom VQE pieces
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import Statevector, SparsePauliOp

# Qiskit Nature + PySCF for electronic Hamiltonian
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper

from pyscf import gto, dft as pyscf_dft

# CoRE-MOF + pymatgen
import CoRE_MOF
from pymatgen.core import Structure
from pymatgen.core.periodic_table import Element


#######################################################################
# USER SETTINGS
#######################################################################
DATASET = "2019-ASR"
MOF_ID = "MAGBAZ_clean"   # you can pick another CoRE 2019 ID if you want

CLUSTER_RADIUS = 2.0      # Å around first metal
MAX_CANDIDATES = 4        # how many QUBO configs to carry into DFT/VQE

# Toy QUBO hyperparameters
TARGET_LONG_FRACTION = 0.5  # roughly how many "1" bits we want
DIST_CUTOFF = 3.0           # Å: connect sites whose distance < cutoff in QUBO graph

# Toy mapping bitstring -> geometry scaling
GEOM_SCALE_ALPHA = 0.05     # max ±5% isotropic scaling around the cluster center

# DFT / VQE basis
BASIS = "lanl2dz"           # safer for heavier metals than sto-3g / def2-svp


#######################################################################
# Helper: build cluster from CoRE-MOF structure
#######################################################################
def get_core_mof_structure(dataset, mof_id):
    struct = CoRE_MOF.get_structure(dataset, mof_id)
    if not isinstance(struct, Structure):
        raise TypeError("CoRE_MOF.get_structure did not return a pymatgen Structure.")
    return struct


def find_first_metal_index(struct):
    for i, site in enumerate(struct):
        elem = Element(site.species_string)
        if elem.is_metal:
            return i
    return 0


def build_cluster_from_structure(struct, radius, center_index=None):
    if center_index is None:
        center_index = find_first_metal_index(struct)
    center = struct[center_index].coords

    species = []
    coords = []
    for site in struct:
        r = np.linalg.norm(site.coords - center)
        if r <= radius:
            species.append(site.species_string)
            coords.append(site.coords)

    coords = np.array(coords, dtype=float)
    return species, coords, center


#######################################################################
# STEP 1: Use cluster geometry to define a QUBO
#######################################################################
def build_mtv_qubo_from_cluster(coords, target_fraction=0.5, dist_cutoff=3.0):
    """
    Encode MTV-like binary choices on each cluster atom:

      - We treat each atom in the cluster as a "site" i.
      - For each site, x_i ∈ {0,1} (two virtual linker types A/B).
      - Composition: sum(x_i) ≈ target_fraction * N
      - Balance: edges between atoms whose distance < dist_cutoff:
                 we penalize x_i == x_j, i.e. favor alternation.

    This yields a QuadraticProgram, QUBO, and Ising operator.
    """
    num_sites = len(coords)
    qp = QuadraticProgram("mtv_cluster_qp")

    bits = [qp.binary_var(f"x_{i}") for i in range(num_sites)]

    # Composition term: (sum x_i - target)^2
    target_long = target_fraction * num_sites
    qp.minimize(constant=0.0)

    # We'll add composition and balance as explicit quadratic objective
    #   composition part: (Σ x_i - target)^2
    linear = {b.name: 0.0 for b in bits}
    quadratic = {}

    # Composition: (Σ x_i)^2 - 2 target Σ x_i + target^2
    # (target^2 is constant, skip)
    # (Σ x_i)^2 = Σ x_i + 2 Σ_{i<j} x_i x_j
    for i in range(num_sites):
        vi = bits[i].name
        linear[vi] += 1.0 - 2.0 * target_long  # from Σ x_i - 2 target x_i
        for j in range(i + 1, num_sites):
            vj = bits[j].name
            key = (vi, vj)
            quadratic[key] = quadratic.get(key, 0.0) + 2.0  # from Σ_{i<j} 2 x_i x_j

    # Balance term: for each edge (i,j) with distance < cutoff:
    # cost_edge = (x_i + x_j - 1)^2, minimal when x_i != x_j.
    # Expand: = 1 - x_i - x_j + 2 x_i x_j (constant dropped)
    edges = []
    for i in range(num_sites):
        for j in range(i + 1, num_sites):
            d_ij = np.linalg.norm(coords[i] - coords[j])
            if d_ij <= dist_cutoff:
                edges.append((i, j))
                vi, vj = bits[i].name, bits[j].name
                linear[vi] += -1.0
                linear[vj] += -1.0
                key = (vi, vj)
                quadratic[key] = quadratic.get(key, 0.0) + 2.0

    qp.minimize(linear=linear, quadratic=quadratic)

    # Convert to QUBO → Ising
    to_qubo = QuadraticProgramToQubo()
    qubo = to_qubo.convert(qp)
    ising_op, offset = to_ising(qubo)

    return qp, qubo, ising_op, offset, edges


#######################################################################
# Custom VQE core (parameter-shift gradient on SparsePauliOp)
#######################################################################
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import Statevector, SparsePauliOp
import numpy as np

def build_hwe_ansatz(num_qubits, depth):
    """
    Simple hardware-efficient ansatz:
      - depth layers of [Ry on each qubit] + CZ ladder
    Returns:
      qc         : QuantumCircuit
      param_list : list of Parameter objects (ordered)
    """
    theta = ParameterVector("θ", length=num_qubits * depth)
    qc = QuantumCircuit(num_qubits)
    k = 0
    for d in range(depth):
        # single-qubit layer
        for q in range(num_qubits):
            qc.ry(theta[k], q)
            k += 1
        # entangling CZ ladder
        for q in range(num_qubits - 1):
            qc.cz(q, q + 1)
    return qc, list(theta)


def expectation_statevector(ham: SparsePauliOp,
                            qc: QuantumCircuit,
                            param_list,
                            params: np.ndarray) -> float:
    """
    Energy E(θ) via exact Statevector, using assign_parameters
    and an explicit mapping from Parameters to values.
    """
    params = np.array(params, dtype=float)
    param_dict = dict(zip(param_list, params))
    bound = qc.assign_parameters(param_dict, inplace=False)
    sv = Statevector.from_instruction(bound)
    return float(np.real(sv.expectation_value(ham)))


def gradient_param_shift(ham,
                         qc,
                         param_list,
                         params,
                         shift=np.pi/2):
    """
    Parameter-shift gradient:
      dE/dθ_j = 0.5 [E(θ_j + s) - E(θ_j - s)]
    """
    params = np.array(params, dtype=float)
    grad = np.zeros_like(params)

    for j in range(len(params)):
        plus = params.copy()
        minus = params.copy()
        plus[j] += shift
        minus[j] -= shift

        e_plus = expectation_statevector(ham, qc, param_list, plus)
        e_minus = expectation_statevector(ham, qc, param_list, minus)
        grad[j] = 0.5 * (e_plus - e_minus)

    return grad


def custom_vqe(ham: SparsePauliOp,
               num_qubits: int,
               depth: int = 2,
               lr: float = 0.1,
               maxiter: int = 100,
               seed: int = 0):
    """
    Fully custom VQE:
      - hardware-efficient ansatz
      - gradient descent with parameter-shift
    """
    qc, param_list = build_hwe_ansatz(num_qubits, depth)
    n_params = len(param_list)

    rng = np.random.default_rng(seed)
    params = rng.uniform(0, 2 * np.pi, size=n_params)

    E = expectation_statevector(ham, qc, param_list, params)
    for it in range(maxiter):
        grad = gradient_param_shift(ham, qc, param_list, params)
        params -= lr * grad
        E = expectation_statevector(ham, qc, param_list, params)

        if it % 10 == 0 or it == maxiter - 1:
            print(f"[VQE] iter {it:3d} E = {E:.6f} |grad| = {np.linalg.norm(grad):.3e}")

    # return param_list as well so we can reuse it later
    return E, params, qc, param_list




from qiskit.quantum_info import Statevector  # make sure this import exists

def top_bitstrings_from_params(qc, param_list, params, max_candidates=4):
    """
    Given an ansatz + optimized params, get the top-k most probable bitstrings.
    """
    params = np.array(params, dtype=float)
    param_dict = dict(zip(param_list, params))
    bound = qc.assign_parameters(param_dict, inplace=False)

    sv = Statevector.from_instruction(bound)
    probs = np.abs(sv.data) ** 2

    idx_sorted = np.argsort(probs)[::-1]
    candidates = []
    for idx in idx_sorted[:max_candidates]:
        p = probs[idx]
        if p < 1e-3:
            continue
        # index -> bitstring, then reverse so qubit 0 is leftmost
        bitstring = format(idx, f"0{qc.num_qubits}b")[::-1]
        candidates.append((bitstring, p))
    return candidates



#######################################################################
# Mapping bitstring -> scaled cluster (toy!)
#######################################################################
def apply_pattern_to_cluster(coords, pattern_bits, alpha=GEOM_SCALE_ALPHA):
    """
    Very toy mapping:

    Given a bitstring pattern [0/1,...] of length L, we compute
      f = (#1)/L
      scale = 1 + alpha * (f - 0.5)
    and isotropically scale the cluster around its centroid by 'scale'.
    """
    coords = np.array(coords, dtype=float)
    pattern = np.array(pattern_bits, dtype=int)
    if len(pattern) == 0:
        f = 0.5
    else:
        f = pattern.mean()

    scale = 1.0 + alpha * (f - 0.5)
    center = coords.mean(axis=0)
    new_coords = center + scale * (coords - center)
    return new_coords, scale


#######################################################################
# DFT on a cluster
#######################################################################
def make_pyscf_mol(species, coords, basis=BASIS):
    atom_str = "; ".join(
        f"{s} {x:.8f} {y:.8f} {z:.8f}"
        for s, (x, y, z) in zip(species, coords)
    )
    mol = gto.Mole()
    mol.atom = atom_str
    mol.basis = basis
    mol.unit = "Angstrom"
    mol.build()
    return mol


def dft_energy_cluster(species, coords, xc="pbe", basis=BASIS):
    mol = make_pyscf_mol(species, coords, basis=basis)
    mf = pyscf_dft.RKS(mol)
    mf.xc = xc
    e_tot = mf.kernel()
    return e_tot  # Hartree


#######################################################################
# Electronic VQE on a cluster (custom VQE on qubit Hamiltonian)
#######################################################################
def build_qubit_hamiltonian_from_cluster(species, coords, basis=BASIS):
    """
    Use PySCFDriver (Qiskit Nature) to get the second-quantized
    electronic Hamiltonian, then map to qubits (SparsePauliOp).
    """
    atom_str = "; ".join(
        f"{s} {x:.8f} {y:.8f} {z:.8f}"
        for s, (x, y, z) in zip(species, coords)
    )

    driver = PySCFDriver(
        atom=atom_str,
        basis=basis,
        unit=DistanceUnit.ANGSTROM,
        charge=0,
        spin=0,
    )
    es_problem = driver.run()

    # second_q_ops()[0] should be the main electronic Hamiltonian
    second_q_ops = es_problem.second_q_ops()
    second_h = second_q_ops[0]

    mapper = JordanWignerMapper()
    qubit_ham = mapper.map(second_h)  # SparsePauliOp

    # NOTE: we are ignoring the explicit nuclear repulsion energy here.
    # For comparing *patterns* at fixed nuclear geometry, this is mostly
    # an overall shift. If you want total energy, add es_problem.hamiltonian.nuclear_repulsion_energy
    return qubit_ham


def vqe_energy_cluster(species, coords, basis=BASIS,
                       depth=1, lr=0.1, maxiter=50, seed=123):
    qubit_h = build_qubit_hamiltonian_from_cluster(species, coords, basis=basis)
    num_qubits = qubit_h.num_qubits

    E, params, qc = custom_vqe(qubit_h, num_qubits,
                               depth=depth, lr=lr,
                               maxiter=maxiter, seed=seed)
    return E  # electronic part only (see note above)


#######################################################################
# MAIN PIPELINE
#######################################################################
if __name__ == "__main__":
    # --------------------------------------------------------------
    # STEP 1: Take .cif (CoRE-MOF), build cluster, encode QUBO
    # --------------------------------------------------------------
    struct = get_core_mof_structure(DATASET, MOF_ID)
    print(f"Loaded {MOF_ID} from {DATASET} with {len(struct.sites)} atoms.")

    species, coords, center = build_cluster_from_structure(struct, CLUSTER_RADIUS)
    print(f"Cluster radius {CLUSTER_RADIUS} Å → {len(species)} atoms.")
    print("Cluster elements:", sorted(set(species)))

    qp, qubo, ising_op, offset, edges = build_mtv_qubo_from_cluster(
        coords,
        target_fraction=TARGET_LONG_FRACTION,
        dist_cutoff=DIST_CUTOFF,
    )

    print("\n=== QUBO (from cluster) ===")
    print(qp.prettyprint())
    print("Ising operator qubits:", ising_op.num_qubits)
    print("Number of QUBO edges:", len(edges))

    # --------------------------------------------------------------
    # STEP 2: Solve QUBO with *custom* VQE (no Qiskit VQE class)
    # --------------------------------------------------------------
    print("\n=== Custom VQE on QUBO Ising Hamiltonian ===")
    num_qubits_qubo = ising_op.num_qubits
    E_qubo, theta_qubo, qc_qubo, param_list_qubo = custom_vqe(
    ham=ising_op,
    num_qubits=num_qubits_qubo,
    depth=2,
    lr=0.1,
    maxiter=80,
    seed=42,)

    print(f"Final QUBO VQE energy (incl. Ising op only) = {E_qubo + offset:.6f}")

    # --------------------------------------------------------------
    # STEP 3: Get top 3–4 classical configurations from VQE
    # --------------------------------------------------------------
    candidates = top_bitstrings_from_params(
    qc_qubo,
    param_list_qubo,
    theta_qubo,
    max_candidates=MAX_CANDIDATES,)

    print("\nTop QUBO bitstrings from VQE (pattern, prob):")
    for bs, p in candidates:
        print(f"  {bs}  p={p:.3f}")

    # Turn bitstrings into patterns (truncate to num_sites = len(coords))
    num_sites = len(coords)
    patterns = []
    for bs, p in candidates:
        bs_rev = bs[::-1]  # make bit 0 = site 0
        patt = [int(b) for b in bs_rev[:num_sites]]
        patterns.append((patt, p))

    # Also define the "original CIF cluster" as a baseline pattern
    # (here just the unscaled geometry, so pattern not used)
    baseline_coords = coords.copy()
    baseline_dft = dft_energy_cluster(species, baseline_coords, xc="pbe", basis=BASIS)
    print(f"\nBaseline DFT energy (original CIF cluster) = {baseline_dft:.8f} Ha")

    # --------------------------------------------------------------
    # STEP 4: For each MTV pattern, apply toy mapping → DFT energy
    # --------------------------------------------------------------
    print("\n=== DFT energies for MTV patterns ===")
    dft_results = []
    for idx, (pattern, prob) in enumerate(patterns):
        mutated_coords, scale = apply_pattern_to_cluster(coords, pattern, alpha=GEOM_SCALE_ALPHA)
        E_dft = dft_energy_cluster(species, mutated_coords, xc="pbe", basis=BASIS)
        dft_results.append((idx, pattern, prob, scale, E_dft))
        print(f"Pattern #{idx}: scale={scale:.4f}, prob={prob:.3f}, E_DFT={E_dft:.8f} Ha")

    if dft_results:
        best_dft = min(dft_results, key=lambda x: x[4])
        print("\nBest by DFT among MTV candidates:")
        print(f"  idx={best_dft[0]}, scale={best_dft[3]:.4f}, E_DFT={best_dft[4]:.8f} Ha")
    else:
        best_dft = None
        print("No MTV candidates had significant probability.")

    # --------------------------------------------------------------
    # STEP 5: For the same patterns, run *custom* VQE on electronic H
    # --------------------------------------------------------------
    print("\n=== Electronic VQE energies for MTV patterns (custom VQE) ===")
    vqe_results = []
    for idx, (pattern, prob) in enumerate(patterns):
        mutated_coords, scale = apply_pattern_to_cluster(coords, pattern, alpha=GEOM_SCALE_ALPHA)
        E_vqe_el = vqe_energy_cluster(
            species,
            mutated_coords,
            basis=BASIS,
            depth=1,         # keep shallow; cluster can be big
            lr=0.1,
            maxiter=30,
            seed=idx + 1,
        )
        vqe_results.append((idx, pattern, prob, scale, E_vqe_el))
        print(f"Pattern #{idx}: scale={scale:.4f}, prob={prob:.3f}, E_VQE_el={E_vqe_el:.8f} Ha (electronic part)")

    if vqe_results:
        best_vqe = min(vqe_results, key=lambda x: x[4])
        print("\nBest by VQE among MTV candidates:")
        print(f"  idx={best_vqe[0]}, scale={best_vqe[3]:.4f}, E_VQE_el={best_vqe[4]:.8f} Ha (electronic only)")
    else:
        best_vqe = None
        print("No MTV candidates for VQE.")

    # --------------------------------------------------------------
    # STEP 6: Compare best DFT / best VQE vs original CIF cluster
    # --------------------------------------------------------------
    print("\n=== Comparison to original CIF cluster ===")

    print(f"Original CIF cluster DFT energy: {baseline_dft:.8f} Ha")

    if best_dft is not None:
        dE_dft = best_dft[4] - baseline_dft
        print(f"Best MTV by DFT: pattern idx={best_dft[0]}, ΔE_DFT = {dE_dft:.8f} Ha")

    if best_vqe is not None:
        # NOTE: we don't have total energy with nuclear repulsion here,
        # only electronic part. So we compare *relative* VQE energies
        # across MTV patterns, not directly to baseline DFT.
        print("Best MTV by VQE is reported in electronic energy only;")
        print("to compare to DFT in absolute terms you'd need to add nuclear repulsion.")

    print("\nDone.")


Loaded MAGBAZ_clean from 2019-ASR with 80 atoms.
Cluster radius 2.0 Å → 4 atoms.
Cluster elements: ['O', 'Zn']

=== QUBO (from cluster) ===
Problem name: mtv_cluster_qp

Minimize
  4*x_0*x_1 + 4*x_0*x_2 + 4*x_0*x_3 + 2*x_1*x_2 + 2*x_1*x_3 + 2*x_2*x_3 - 6*x_0
  - 4*x_1 - 4*x_2 - 4*x_3

Subject to
  No constraints

  Binary variables (4)
    x_0 x_1 x_2 x_3

Ising operator qubits: 4
Number of QUBO edges: 3

=== Custom VQE on QUBO Ising Hamiltonian ===
[VQE] iter   0 E = 0.550231 |grad| = 2.675e+00
[VQE] iter  10 E = -1.464427 |grad| = 3.890e-01
[VQE] iter  20 E = -1.497290 |grad| = 8.275e-02
[VQE] iter  30 E = -1.499426 |grad| = 2.724e-02
[VQE] iter  40 E = -1.499785 |grad| = 1.437e-02
[VQE] iter  50 E = -1.499909 |grad| = 9.086e-03
[VQE] iter  60 E = -1.499961 |grad| = 5.916e-03
[VQE] iter  70 E = -1.499983 |grad| = 3.868e-03
[VQE] iter  79 E = -1.499992 |grad| = 2.640e-03
Final QUBO VQE energy (incl. Ising op only) = -5.999992

Top QUBO bitstrings from VQE (pattern, prob):
  0101  p=0.

KeyboardInterrupt: 

#######################################################################
# Visualization: Scaling Function for QUBO Pattern Mapping
#######################################################################

This section visualizes the toy mapping from bitstring patterns to geometry scaling.
The scaling factor is a linear function of the bitstring fraction (proportion of 1s):
- `scale = 1.0 + alpha * (fraction - 0.5)`
- When `fraction = 0` (all 0s): cluster shrinks by `alpha/2`
- When `fraction = 1` (all 1s): cluster expands by `alpha/2`
- When `fraction = 0.5` (balanced): no scaling (scale = 1.0)

In [ ]:
import matplotlib.pyplot as plt

# Plot the scaling function for various alpha values
fractions = np.linspace(0, 1, 100)
alphas = [0.02, 0.05, 0.1, 0.2]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Scaling factor vs fraction for different alpha values
for alpha in alphas:
    scales = 1.0 + alpha * (fractions - 0.5)
    ax1.plot(fractions, scales, label=f"α = {alpha}", linewidth=2)

ax1.axhline(y=1.0, color='k', linestyle='--', linewidth=0.8, alpha=0.5, label='No scaling')
ax1.set_xlabel('Bitstring Fraction (proportion of 1s)', fontsize=12)
ax1.set_ylabel('Scaling Factor', fontsize=12)
ax1.set_title('QUBO Pattern → Geometry Scaling', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Plot 2: Energy shift (toy) vs scaling for current alpha
alpha_current = GEOM_SCALE_ALPHA
scales_current = 1.0 + alpha_current * (fractions - 0.5)

# Toy energy surface: quadratic with minimum at fraction=0.5
# E(s) = (s - 0.5)^2 * 10  (arbitrary scaling)
energy_landscape = (fractions - 0.5) ** 2 * 10

ax2.fill_between(fractions, energy_landscape, alpha=0.3)
ax2.plot(fractions, energy_landscape, 'b-', linewidth=2, label='Toy energy landscape')
ax2.axvline(x=0.5, color='g', linestyle='--', linewidth=1.5, alpha=0.7, label='Balanced (frac=0.5)')
ax2.set_xlabel('Bitstring Fraction (proportion of 1s)', fontsize=12)
ax2.set_ylabel('Energy (arbitrary units)', fontsize=12)
ax2.set_title(f'Toy Energy Landscape (α = {alpha_current})', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

print(f"Scaling function: scale = 1.0 + α × (fraction - 0.5)")
print(f"Current α = {GEOM_SCALE_ALPHA}")
print(f"  fraction = 0.0 → scale = {1.0 + GEOM_SCALE_ALPHA * (0.0 - 0.5):.4f} (shrink by {GEOM_SCALE_ALPHA*50:.1f}%)")
print(f"  fraction = 0.5 → scale = {1.0 + GEOM_SCALE_ALPHA * (0.5 - 0.5):.4f} (no scaling)")
print(f"  fraction = 1.0 → scale = {1.0 + GEOM_SCALE_ALPHA * (1.0 - 0.5):.4f} (expand by {GEOM_SCALE_ALPHA*50:.1f}%)")
